In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils import get_completion

tokenizer = AutoTokenizer.from_pretrained("kreimben/CodeMind-gemma", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("kreimben/CodeMind-gemma")

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model = model.to('cuda')

In [3]:
res = get_completion(
    'i dont know about word break problem. please give me a approch about this problem.',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096
)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
res = res.split('model')[-1]

In [9]:
from IPython.display import display, Markdown

res = res.replace('\\\\', '\\')

display(Markdown(res))

\n\n**Approach 1: Iterative Approach**\n\n**Step 1: Preprocess Words**\n\n- Convert all words to lowercase and remove any leading or trailing spaces.\n- Split the words into chunks by spaces or using specific markers [ e.g \"/\n\"]\n\n**Step 2: Iterate on Chunks**\n\n- The loop iterates over the preprocessed chunks.\n- For each chunk, find the maximum number of contiguous words in the chunk that are part of words in our list. This can be calculated using the `max()` method on the chunk.\n- If a chunk has at least one word that can be part of the words listed in `words`, it is part of a "word break".\n\n**Example:**\n\n```\nwords = [\'abcdefghijk\', \'ace\']\nwordBreak3(words, "ace", "ace")\n```\n\n* Preprocess words:\n\nThe words list is an array of strings, and we can preprocess them by converting them to lowercase and removing any leading or trailing spaces.\n```\nfor word in words:\n    word = word.lower() + " " + word \n```\n\n\n* Iterate on chunks and find the maximum continuous word count:\n\nThe chunk "ace" can be processed using "abcdefghijk", and so on. The maximum continuous word count can be found using the `max()` method on the chunk: \n\n```\nif chunk[1:] in words:\n    continue       # skip subsequent words in chunk that are already part of words\nmax_continuous = max([max([word[1:] for word in words if word[:word[0]] in chunk[1:word[0]+1]]), chunk[1:chunk[-1]]]) \nif max_continuous > 1:\n    wordCount += 1\n```\n\nThe process then repeats on each subsequent chunk and continues until it finds a word of wordCount words length or more in the list.\n\n**Time Complexity:**\nO(n^2)\n\n* Preprocessing the words is O(n) Time, and the loop iterates over words * chunks. The code runs in linear time as long as there are no character limitations on words or chunks. In practice, we can use a more efficient data structure to store words (e.g. treemap) which will result in **O(n log n)** Time complexity, as we will only loop through words once.\n\n**Space Complexity:**\nO(k)\n\n* Time: O(len(words * chunks))\n* Space: O(wordCount),\n  where wordCount is up to O(k) where k is the number of words.\n\n**Runtime:**\nThe code uses `len(words)` to find the number of words in the `words` list, and then adds a constant number (`wordCount += 1`) to `wordCount` to get the sum of wordCount as a constant number. The maximum wordCount is then stored in `result`. Therefore, each loop iterates through `len(words)` words. Consequently, the time complexity is O(n^2). We can use a simpler approach on the backtrack loop to save some space. If there is a loop backtracking the same space of wordCount times (i.e. for each word in words) then the run time reduction is O(wordCount)^2 = O(n^2) but the memory complexity will never be O(n^2) because it is O(wordCount).

**Complexity:**\n\nTime complexity:\nO(m^2), where words is the length of the word array\n\nSpace complexity:\nO(m), where m is the length of the word array\n\n**Idea 2: Tabulation Approach**\n\nIn this approach, pre-process the words to find potential word breaks.\n\n* Define `dp` as a 2D array with `wordList` rows and `wordCount` columns, initialized with all values set to `0`. `dp[0][0]` is the boolean value for "word 1" in `wordList`.\n* Initialize the first row and column of `dp` with the boolean value for the first word in `wordList` and wordCount equal to 1.\n* Iterate through the word list.\n    * If `dp[wordIndex][wordCount] == 1, proceed to calculate word break for the next word:\n        * Iterate across the next consecutive words in the list:\n            * For each consecutive word, check if it starts with the current word and has a word length of `wordCount + 1`, and save the current word in the result:\n              * If there are no more words to check, then we have a word break in the current wordList:\n                * Update the `dp[endWordWordIndex][result.length]` as 1 and move back to process the `wordList` and the result.\n* At the end of the loop, `dp` contains the true value for each cell.\n\n`wordList` array is the original word array, `wordCount` is the sum of words lengths in the word array, and `result` stores the boolean value for each word in the output array. `dp` array has the result: `DP[word][wordCount]`.  \n\n```py\nclass Solution:\n    def wordBreak(self, words: List[str], breakpoint: str) -> bool:\n        # Preprocess words\n\n        words = ["a","abcd"]\n        # dp[wordIndex][wordCount] == 1 means we have a word break in this word, which contains the next word \n        # dp[wordIndex][wordCount] is a 2-dimensional array whose row length is word_size and column length is word_count\n        dp = [[0] * wordCount for wordIndex in range(word_size)]\n        \n        # Initialize dp values\n        for wordIndex in range(len(words)):         \n            dp[wordIndex][0] = 1 \n        \n        # Iterate through words\n        for idx in range(1, len(words)):    \n            # Check if we have a word break\n            for wordIndex in range(idx):\n                if dp[wordIndex][wordCount] == 1:\n                    break\n                    \n            # The word is already part of words\n            \n\n            # Iterate through consecutive words in words list\n            for wordIndex in range(idx, idx + vocab_size):\n                # Check if we have a word break\n                if dp[idx + wordIndex][wordCount] == 0:\n                    break\n                    \n                # We have a word break, process the next word and store the result\n                for j in range(wordIndex + 1, idx + vocab_size):\n                    # Add the subsequent word to the result\n                    dp[wordIndex + wordJ][wordCount + wordJCount] = 1\n                    # We have a word break, break out of this outer loop\n                    break\n                \n        \n        # Process the last word\n        return dp[-1][-1]\n``` 
\n\n**Time Complexity:**\nBest: O(n), Average: O(n^2), Worst: O(n^2)\n\n**Space Complexity:**     \n\n**For more detail, refer this article :**\n\n**Code for Tabulation Approach**\n```py\nclass Solution:\n    def wordBreak(self, words: List[str], breakpoint: str) -> bool:\n        wordList = ["a","abcd"]\n        \n        # Initialize dp values\n        dp = [[1] * len(word) for wordIndex in range(len(word))]\n        \n        # Initialize dp values\n        for wordIndex in range(len(wordList)):         \n            dp[wordIndex][0] = 1 \n        \n        # Iterate through words\n        for idx in range(1, len(words)):    \n            # iterate through words in words\n            for wordIndex in range(idx, len(words)):\n                ## for wordIndex in range(idx, wordIndex + vocab_size): \n                # iterate through words in words to process the wordIndex of words\n                # if wordIndex != wordIndex + vocab_size\n                # dp[wordIndex][wordCount] == 1 means we have a word break in word, which contains the subsequent word\n                if dp[wordIndex][0] == 1:\n                    continue\n                \n                # we already have a word break, we can now process the forthcoming word \n                for j in range(wordIndex, wordIndex + vocab_size):\n                    # break out of the word array so the subsequent words are ignored\n                    if wordIndex == wordIndex + vocab_size - 1:\n                        break\n                                \n                    # check if we have a word break for the subsequent word\n                    if dp[wordIndex + j][0] == 0:\n                        continue\n                    # we have a word break for the subsequent word, check for if we are going to update this result\n                    for k in range(j + 1, wordIndex + vocab_size):\n                        dp[wordIndex + k][wordCount + vocab_size] = 1\n                    # if there are no more words, move back the result back to the processing\n                    # move back to process the wordArray\n                    wordIndex = wordIndex + vocab_size\n        # Process the last word\n        return dp[-1][-1]\n``` \n\n**Please upvote this post if understand the provided code. Thank you!**\n\n \n**Follow my [Gitbook]() for more algorithms and data structures tutorials and notes.**\n\n**Any questions are welcomed.** \n\n\n```py\nO(m*n^2) Time Complexity, O(m*n^2) Space Complexity\n```